In [13]:
import pandas as pd
import numpy as np
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight
from tsfresh.feature_selection import select_features


In [14]:
raw_data_prep_dir = '../raw_data/preprocessed/'

In [15]:
file_list = pd.read_csv(raw_data_prep_dir  + "file_list.csv")
file_list

,resource_type,id,study_id,condition,disease_comment,age_at_diagnosis,age,height,weight,gender,handedness,appearance_in_kinship,appearance_in_first_grade_kinship,effect_of_alcohol_on_tremor,label
0,patient,1,PADS,Healthy,-,56,56,173,78,male,right,True,True,Unknown,0
1,patient,2,PADS,Other Movement Disorders,Left-Sided resting tremor and hypokinesia with...,69,81,193,104,male,right,False,NaN,No effect,2
2,patient,3,PADS,Healthy,-,45,45,170,78,female,right,False,NaN,Unknown,0
3,patient,4,PADS,Parkinson's,IPS akinetic-rigid type,63,67,161,90,female,right,False,NaN,No effect,1
4,patient,5,PADS,Parkinson's,IPS tremordominant type,65,75,172,86,male,left,False,NaN,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,patient,465,PADS,Parkinson's,IPS mixed type,62,65,175,80,male,right,True,False,No effect,1
465,patient,466,PADS,Healthy,-,84,84,172,74,female,right,True,True,No effect,0
466,patient,467,PADS,Parkinson's,"Essential Tremor, starting IPS tremordominant ...",55,57,190,100,male,right,False,NaN,Improvement,1
467,patient,468,PADS,Parkinson's,IPS mixed type,73,76,198,118,male,right,False,NaN,No effect,1


In [16]:
n_id = len(file_list['id'])
n_id

469

In [17]:
file_list[file_list['id']==5]['label']

4    1
Name: label, dtype: int64

In [18]:
time_data = []
y = []
X_to_split = []
for idx, subject_idx in enumerate(file_list['id']):
    file_name = raw_data_prep_dir + f'movement/{subject_idx:03d}_ml.bin'
    time_idx_data = np.fromfile(file_name, dtype=np.float32).reshape((-1, 976))
    time_data.append(time_idx_data)
    y_idx = file_list[file_list['id']==subject_idx].label.values
    X_id = idx
    y.append(y_idx)
    X_to_split.append(X_id)

X_to_split = np.array(X_to_split)
y = np.array(y).flatten()
print(X_to_split.shape, y.shape)


(469,) (469,)


value_cols = ['x','y','z','speed','acc','ang_vel']
fc = EfficientFCParameters()  # compact, fast set
X = extract_features(df[['traj_id','t'] + value_cols],
                     column_id='traj_id', column_sort='t',
                     default_fc_parameters=fc, disable_progressbar=True)

In [19]:
channels = []
for task in ["Relaxed1", "Relaxed2", "RelaxedTask1", "RelaxedTask2", "StretchHold", "HoldWeight",
                "DrinkGlas", "CrossArms", "TouchNose", "Entrainment1", "Entrainment2"]:
    for device_location in ["LeftWrist", "RightWrist"]:
        for sensor in ["Acceleration", "Rotation"]:
            for axis in ["X", "Y", "Z"]:
                channel = f"{task}_{sensor}_{device_location}_{axis}"
                channels.append(channel)
len(channels)

132

In [20]:
time_data_array = np.array(time_data)
print(time_data_array.shape)

(469, 132, 976)


In [21]:
X_train_idx, X_test_idx, y_train, y_test = train_test_split(X_to_split, y, test_size=0.20, random_state=42, stratify = y)

In [22]:
X_train = time_data_array[X_train_idx]
X_test = time_data_array[X_test_idx]
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))
X = time_data_array
X.shape[0]

375 94
375 94


469

In [23]:
import numpy as np
import pandas as pd
from tsfresh.feature_extraction import extract_features, MinimalFCParameters

# shape (N, F, T)
N, F, T = X_test.shape
feature_names = [f"feat_{i}" for i in range(F)]

def make_long_df(batch_data: np.ndarray, id_offset: int = 0) -> pd.DataFrame:

    n_batch = batch_data.shape[0]


    x = np.transpose(batch_data, (0, 2, 1))
    values = x.reshape(-1)

    ids = np.repeat(np.arange(id_offset, id_offset + n_batch), T * F)


    times = np.tile(np.repeat(np.arange(T), F), n_batch)


    kinds_idx = np.tile(np.arange(F), n_batch * T)
    kinds = np.array(feature_names, dtype=object)[kinds_idx]

    return pd.DataFrame({"id": ids, "time": times, "kind": kinds, "value": values})

df_long = make_long_df(X, id_offset=0)


X_features = extract_features(
    df_long,
    column_id="id",
    column_sort="time",
    column_kind="kind",
    column_value="value",
    default_fc_parameters=MinimalFCParameters(),
    n_jobs=0
)

print(X_features.shape)  # (375, num_features)

Feature Extraction: 100%|██████████| 61908/61908 [00:10<00:00, 5669.54it/s]


(469, 1320)


In [24]:
y_series = pd.Series(y, index=X_features.index)
classes = np.unique(y_series)

selected_sets = []
for c in classes:
    y_bin = (y_series == c).astype(int)
    sel_c = select_features(X_features, y_bin, fdr_level=0.05)
    selected_sets.append(set(sel_c.columns))

selected_cols = sorted(set().union(*selected_sets))
X_selected = X_features[selected_cols]
print(f"Selected {len(selected_cols)} / {X_features.shape[1]} features")

Selected 314 / 1320 features


In [25]:
from sklearn.feature_selection import VarianceThreshold

# remove quasi-constant features
vt = VarianceThreshold(threshold=1e-6)
X_selected = pd.DataFrame(
    vt.fit_transform(X_selected),
    index=X_selected.index,
    columns=X_selected.columns[vt.get_support()]
)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.20, random_state=42, stratify = y)

minmaxscaler = MinMaxScaler()
minmaxscaler.fit(X_train)

X_train_trans = minmaxscaler.transform(X_train)
X_test_trans = minmaxscaler.transform(X_test)

pd.DataFrame(X_train_trans)


,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,0.004075,0.000054,0.226005,0.228747,0.731327,0.450843,0.414772,0.175806,0.562207,0.281432,...,2.142127e-05,0.060061,0.088703,0.115247,0.051154,0.051153,0.115247,0.003042,0.082392,0.041903
1,0.004010,0.000053,0.168193,0.170229,0.800054,0.288321,0.288547,0.086449,0.667747,0.640153,...,7.997404e-05,0.096468,0.077263,0.092756,0.055779,0.055775,0.092756,0.003572,0.051505,0.036791
2,0.006388,0.000099,0.276382,0.249449,0.671440,0.503433,0.503001,0.256894,0.748242,0.257188,...,1.425290e-05,0.040180,0.059341,0.072998,0.026209,0.026163,0.072998,0.000908,0.025185,0.046561
3,0.008144,0.000140,0.133638,0.101214,0.841133,0.214526,0.194057,0.040088,0.745806,0.415223,...,3.594777e-04,0.029156,0.031441,0.112550,0.032099,0.032098,0.112550,0.001302,0.083949,0.062924
4,0.011905,0.000249,0.289335,0.292214,0.804992,0.439772,0.419314,0.179607,0.744171,0.558606,...,5.013921e-05,0.069317,0.042945,0.015894,0.039847,0.039623,0.015894,0.001903,0.160811,0.033773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,0.432625,0.189408,0.316518,0.319287,0.626368,0.369471,0.366052,0.137599,0.803068,0.613454,...,1.654213e-01,0.237489,0.304402,0.124579,0.137505,0.137503,0.124579,0.019946,0.202864,0.077226
371,0.004259,0.000057,0.225181,0.228320,0.805127,0.287158,0.285573,0.084721,0.641763,0.475266,...,4.812035e-07,0.030047,0.044375,0.094429,0.019722,0.019714,0.094429,0.000558,0.056673,0.055223
372,0.003530,0.000045,0.287060,0.289949,0.742201,0.421870,0.402289,0.165572,0.799786,0.532212,...,1.603042e-05,0.043940,0.064895,0.085740,0.020540,0.020517,0.085740,0.000597,0.072398,0.033563
373,0.009132,0.000166,0.195594,0.198853,0.804357,0.285929,0.278526,0.080698,0.817654,0.538668,...,2.158440e-04,0.045364,0.042835,0.057989,0.034142,0.034067,0.057989,0.001449,0.043545,0.035155


In [27]:
X_train

,feat_0__standard_deviation,feat_0__variance,feat_100__absolute_maximum,feat_100__maximum,feat_100__minimum,feat_100__root_mean_square,feat_100__standard_deviation,feat_100__variance,feat_101__median,feat_105__median,...,feat_8__variance,feat_90__absolute_maximum,feat_90__maximum,feat_90__mean,feat_90__root_mean_square,feat_90__standard_deviation,feat_90__sum_values,feat_90__variance,feat_93__maximum,feat_95__median
48,0.002326,0.000005,3.493921,3.488036,-3.493921,1.953538,1.798539,3.234743,-0.448186,-0.529173,...,3.132708e-06,0.201972,0.201972,0.000256,0.038707,0.038706,0.249898,0.001498,1.176627,0.002263
12,0.002309,0.000005,2.635758,2.615875,-2.635758,1.261421,1.261421,1.591183,-0.286137,0.019494,...,9.299471e-06,0.318186,0.177248,-0.000504,0.041929,0.041926,-0.492312,0.001758,0.746579,-0.001761
282,0.002946,0.000009,4.241709,3.796590,-4.241709,2.177500,2.173980,4.726189,-0.162543,-0.566254,...,2.377737e-06,0.138511,0.138511,-0.001172,0.021333,0.021301,-1.144352,0.000454,0.380101,0.005930
128,0.003416,0.000012,2.122830,1.587256,-2.122830,0.947159,0.859341,0.738468,-0.166282,-0.324538,...,3.873673e-05,0.103321,0.078210,0.000165,0.025435,0.025435,0.160908,0.000647,1.198311,0.018814
32,0.004423,0.000020,4.433975,4.433975,-2.574095,1.906392,1.817867,3.304641,-0.168792,-0.105234,...,6.157274e-06,0.231518,0.103074,-0.003103,0.030832,0.030676,-3.028829,0.000941,2.268508,-0.004138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,0.117086,0.013709,4.837474,4.837474,-4.804498,1.607008,1.591226,2.531999,-0.078360,-0.021343,...,1.742300e-02,0.768338,0.668178,0.000572,0.098852,0.098851,0.557882,0.009771,2.854037,0.030073
302,0.002375,0.000006,3.481683,3.481683,-2.572420,1.256470,1.248762,1.559407,-0.326034,-0.232703,...,9.273061e-07,0.106165,0.106165,-0.000448,0.016815,0.016809,-0.437098,0.000283,0.818534,0.012750
253,0.002180,0.000005,4.400215,4.400215,-3.358138,1.830154,1.745421,3.046493,-0.083399,-0.145604,...,2.564946e-06,0.150515,0.150515,-0.000742,0.017384,0.017368,-0.723849,0.000302,1.037486,-0.004303
202,0.003680,0.000014,3.042498,3.042498,-2.582024,1.251234,1.218777,1.485417,-0.055964,-0.135728,...,2.360926e-05,0.155060,0.102837,-0.001680,0.026858,0.026806,-1.639646,0.000719,0.635736,-0.003050


In [28]:
classes = np.unique(y_train)
weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

base_model_xgb = XGBClassifier(scale_pos_weight=class_weights)
base_model_xgb.fit(X_train_trans,y_train)

y_pred = base_model_xgb.predict(X_test_trans)


# Metrics
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [08:42:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Balanced Accuracy: 0.5779808959156786

Confusion Matrix:
 [[ 7  8  1]
 [ 3 45  7]
 [ 2 10 11]]

Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.44      0.50        16
           1       0.71      0.82      0.76        55
           2       0.58      0.48      0.52        23

    accuracy                           0.67        94
   macro avg       0.63      0.58      0.60        94
weighted avg       0.66      0.67      0.66        94



In [29]:

xgb = XGBClassifier()

sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)


xgb = XGBClassifier()

xgb.fit(X_train, y_train, sample_weight=sample_weights)

y_pred = xgb.predict(X_test)


# Metrics
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Balanced Accuracy: 0.6338603425559947

Confusion Matrix:
 [[ 8  4  4]
 [ 4 46  5]
 [ 3  7 13]]

Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.50      0.52        16
           1       0.81      0.84      0.82        55
           2       0.59      0.57      0.58        23

    accuracy                           0.71        94
   macro avg       0.64      0.63      0.64        94
weighted avg       0.71      0.71      0.71        94

